In [7]:
import praw
import requests
import pandas as pd

CLIENT_ID = 'BP74HTk3g44RqYHGWnHNaA'
secret_key = 'CvyjdJVmQWuiMAaOqsIC1wCHv4RqlA'

auth = requests.auth.HTTPBasicAuth(CLIENT_ID,secret_key)
data = {
    'grant_type': 'password',
    'username': 'Datahydration',
    'password': 'Br3Yp$F93s-%!pb'
}
headers = {'User-Agent': 'redditapi/1'}
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)
token12 = res.json()['access_token']
headers = {**headers, **{'Authorization': f'bearer {token12}'}}


url = "https://oauth.reddit.com/r/wallstreetbets/"
query_url = url + "best"
best_posts = requests.get(query_url, headers=headers)
best_json = best_posts.json()
best_json

{'kind': 'Listing',
 'data': {'after': 't3_18chog3',
  'dist': 27,
  'modhash': None,
  'geo_filter': None,
  'children': [{'kind': 't3',
    'data': {'approved_at_utc': None,
     'subreddit': 'wallstreetbets',
     'selftext': "Make sure you're in the [WSB Discord](https://discord.gg/wsbverse)!\n\nCheck out our [Earnings Thread](https://reddit.com/r/wallstreetbets/about/sticky?num=2) and [Rules](https://www.reddit.com/r/wallstreetbets/wiki/contentguide). DM the [mod inbox](https://www.reddit.com/message/compose/?to=/r/wallstreetbets)",
     'author_fullname': 't2_bd6q5',
     'saved': False,
     'mod_reason_title': None,
     'gilded': 0,
     'clicked': False,
     'title': 'What Are Your Moves Tomorrow, December 08, 2023',
     'link_flair_richtext': [{'e': 'text', 't': 'Daily Discussion'}],
     'subreddit_name_prefixed': 'r/wallstreetbets',
     'hidden': False,
     'pwls': 7,
     'link_flair_css_class': 'daily',
     'downs': 0,
     'thumbnail_height': None,
     'top_awarde

In [8]:
import json
import requests
url = "https://yahoo-finance127.p.rapidapi.com/price/tsla"
headers = {
    "X-RapidAPI-Key": "11f7a923e1msh3c8c61e1bef42dcp1a3bf6jsn7810d3eb53c7",
    "X-RapidAPI-Host": "yahoo-finance127.p.rapidapi.com"
}
# response = requests.get(url, headers=headers)

# token1 = str(response.json())
#json.dumps(token1, indent=4, sort_keys=True)
yahoo_response = requests.get(url, headers=headers).json()
yahoo_response
# print(json.dumps(data, indent=4, sort_keys=True))

{'symbol': 'TSLA',
 'twoHundredDayAverageChangePercent': {'raw': 0.079507574, 'fmt': '7.95%'},
 'fiftyTwoWeekLowChangePercent': {'raw': 1.383263, 'fmt': '138.33%'},
 'averageAnalystRating': '2.8 - Hold',
 'language': 'en-US',
 'regularMarketDayRange': {'raw': '236.98 - 244.08', 'fmt': '236.98 - 244.08'},
 'earningsTimestampEnd': {'raw': 1706529600,
  'fmt': '2024-01-29',
  'longFmt': '2024-01-29T07:00'},
 'epsForward': {'raw': 3.84, 'fmt': '3.84'},
 'regularMarketDayHigh': {'raw': 244.08, 'fmt': '244.08'},
 'twoHundredDayAverageChange': {'raw': 17.87085, 'fmt': '17.87'},
 'askSize': {'raw': 11, 'fmt': '11', 'longFmt': '11'},
 'twoHundredDayAverage': {'raw': 224.76915, 'fmt': '224.77'},
 'bookValue': {'raw': 16.818, 'fmt': '16.82'},
 'fiftyTwoWeekHighChange': {'raw': -56.65001, 'fmt': '-56.65'},
 'marketCap': {'raw': 771333160960,
  'fmt': '771.333B',
  'longFmt': '771,333,160,960'},
 'fiftyTwoWeekRange': {'raw': '101.81 - 299.29', 'fmt': '101.81 - 299.29'},
 'fiftyDayAverageChange': {'

In [12]:
CLIENT_ID = 'BP74HTk3g44RqYHGWnHNaA'
secret_key = 'CvyjdJVmQWuiMAaOqsIC1wCHv4RqlA'
import requests
import praw
auth = requests.auth.HTTPBasicAuth(CLIENT_ID,secret_key)
data = {
    'grant_type': 'password',
    'username': 'Datahydration',
    'password': 'Br3Yp$F93s-%!pb'
}
headers = {'User-Agent': 'redditapi/1'}
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)
token12 = res.json()['access_token']
headers = {**headers, **{'Authorization': f'bearer {token12}'}}
headers
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

reddit_posts = []
reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=secret_key,
    password = data['password'],
    user_agent=f"testscript by u/{data['username']}",
    username = data['username'],
)
for submission in reddit.subreddit("wallstreetbets").search("Tesla", limit=10, sort="new"):

    reddit_posts.append({"title": submission.title,
                         "upvotes": submission.score,
                         "num_comments": submission.num_comments,
                         "date_utc": submission.created_utc})

reddit_df = pd.DataFrame(reddit_posts)
reddit_df.head()

,title,upvotes,num_comments,date_utc
0,Would this be considered classic Kramer move? ...,14,13,1.701986e+09
1,Where is all the damn gain porn?,4,14,1.701983e+09
2,"Elon Musk goes off on Disney CEO Bob Iger, say...",1437,623,1.701972e+09
3,Mag7 contributed around 84% of the S&P 500's g...,4,6,1.701961e+09
4,Danish pension funds (43 billion USD) starts s...,818,270,1.701904e+09


In [11]:
yahoo_finance = []

yahoo_finance.append({"Symbol": yahoo_response["symbol"],
                        "Day_Range_raw": yahoo_response["regularMarketDayRange"]["raw"],
                        "Market_Price": yahoo_response["regularMarketPrice"]["raw"],
                        "Company_Name": yahoo_response["longName"]})

yahoo_finance_df = pd.DataFrame(yahoo_finance)
yahoo_finance_df.head()


,Symbol,Day_Range_raw,Market_Price,Company_Name
0,TSLA,236.98 - 244.08,242.64,"Tesla, Inc."
